# mDOM Pin Verification

### First, I will parse pin information from the Xilinx file here: https://www.xilinx.com/support/packagefiles/s7packages/xc7s100fgga676pkg.txt

### The xc7s100fgga676 is the FPGA used on the mDOM mainboard.

In [1]:
import urllib

In [2]:
headers = ['Pin',
           'Pin Name',
           'Memory Byte Group',
           'Bank',
           'VCCAUX Group',
           'Super Logic Region',
           'I/O Type',
           'No-Connect']

def parse_pin_file(file):
    '''returns a dict of pin dictionaries
    the key is the pin name
    '''
    for line in file:
        spl = line.decode().split()
        try:
            if spl[0] == 'Pin':
                break
        except IndexError:
            pass


    pins = {}
    for line in file:
        spl = line.decode().split()
        try:
            pin = {header: spl[i+1] for i, header in enumerate(headers[1:])}
        except IndexError:
            continue
        pins[spl[0]] = pin

    return pins

### Here I record bank voltage levels from the schematic located here:
https://www-zeuthen.desy.de/~sulanke/Projects/ICECUBE/mDOM/mainboard/schematic/mDOM_mb_1.pdf 

In [3]:
bank_Vs = {0: 1.8, 13: 1.8, 14: 1.8, 15: 1.8, 16: 1.8, 33: 1.35, 34: 1.35, 35: 1.8, 36: 3.3}

In [4]:
with urllib.request.urlopen('https://www.xilinx.com/support/packagefiles/s7packages/xc7s100fgga676pkg.txt') as f:
    pins = parse_pin_file(f)

In [5]:
print(pins['AC14'])
print(pins['R14'])

{'Pin Name': 'DONE_0', 'Memory Byte Group': 'NA', 'Bank': '0', 'VCCAUX Group': 'NA', 'Super Logic Region': 'NA', 'I/O Type': 'CONFIG', 'No-Connect': 'NA'}
{'Pin Name': 'DXP_0', 'Memory Byte Group': 'NA', 'Bank': '0', 'VCCAUX Group': 'NA', 'Super Logic Region': 'NA', 'I/O Type': 'CONFIG', 'No-Connect': 'NA'}


### Now I will parse the XDC file that Kalle sent me, which specifies the pin assignments.

In [6]:
def parse_xdc_file(file):
    '''returns a list of pin dictionaries
    '''

    pins = []

    for line in file:
        spl = line.split()
        pin_dict = {}
        for i, word in enumerate(spl):
            if word == 'PACKAGE_PIN':
                pin_dict['Pin'] = spl[i+1]
            elif word == '[get_ports':
                port = spl[i+1].replace('{', '').replace('}','').replace(']', '')
                differential = port.endswith('P') or port.endswith('M') or port.endswith('N')
                
                # ports ending with "SEN" or "DIN" end with N but are not differential...
                # More standardized port names (e.g. _N/_M) would help here.
                differential &= not port.endswith('SEN')
                differential &= not port.endswith('DIN')
                
                # same with LDM, UDM
                differential &= not port.endswith('LDM')
                differential &= not port.endswith('UDM')
                
                differential &= not port.endswith('TRIG_IN')
                
                pin_dict['Port'] = port
                pin_dict['Differential'] = differential

        pins.append(pin_dict)

    return pins

In [7]:
with open('mDOM_mb_1_jun29.xdc') as f:
    ports = parse_xdc_file(f)

In [8]:
print(ports[0])
print(ports[-1])

{'Pin': 'AC19', 'Port': 'ADC0_CLOCK_P', 'Differential': True}
{'Pin': 'A8', 'Port': 'TRIG_OUT', 'Differential': False}


### I will now sort ports into their banks, keeping differential and single-ended IO separate

In [9]:
banks = {f'{bank}': {'differential_pins': [], 'single_ended_pins': []} for bank in bank_Vs}
for port in ports:
    pin = pins[port['Pin']]
    # merge pin and port dicts
    for key, val in port.items():
        pin[key] = val
    
    bank_dict = banks.setdefault(pin['Bank'], )
    if port['Differential']:
        bank_dict['differential_pins'].append(pin)
    else:
        bank_dict['single_ended_pins'].append(pin)

### Display pins organized by bank and by differential / single-ended

In [10]:
from IPython.display import display, Markdown, Latex

In [11]:
display(Markdown(f'## Single-ended ports'))
for bank_name, pins in banks.items():
    display(Markdown(f'### Bank {bank_name}, {bank_Vs[int(bank_name)]}V'))
    
    single_ended = pins['single_ended_pins']
    single_ended = sorted(single_ended, key=lambda x: x['Pin'])
    
    for pin in single_ended:        
        display(Markdown(f'**{pin["Pin"]}** *{pin["Pin Name"]}* **{pin["Port"]}**'))

## Single-ended ports

### Bank 0, 1.8V

### Bank 13, 1.8V

**AE17** *IO_25_13* **ADC0_SEN**

**AF24** *IO_0_13* **ADC1_SEN**

...

**E5** *IO_L17P_T2_36* **LIS3MDL_DRDY**

**E8** *IO_L7N_T1_36* **QOSC_VC_ENA**

**F10** *IO_L10P_T1_36* **DAC1_DIN**

**F8** *IO_L7P_T1_36* **QOSC_SCL**

**F9** *IO_L10N_T1_36* **DAC2_SCLK**

### Now I'll look at differential pairs

In [12]:
display(Markdown(f'## Differential ports'))

all_properly_paired = True

for bank_name, pins in banks.items():
    display(Markdown(f'### Bank {bank_name}, {bank_Vs[int(bank_name)]}V'))
    display(Markdown('--------'))
    
    diff_pins = pins['differential_pins']
    diff_pins = sorted(diff_pins, key=lambda x: x['Port'])
    
    # there must be an even number of differential pins
    if len(diff_pins) % 2 != 0:
        for pin in diff_pins:
            display(Markdown(f'**{pin["Pin"]}** *{pin["Pin Name"]}* **{pin["Port"]}**'))
        raise RuntimeError('Odd number of differntial pins!')
    
    for pin_p, pin_n in zip(diff_pins[1::2], diff_pins[::2]):
        p_name = pin_p['Pin Name']
        n_name = pin_n['Pin Name']
        
        split_ind = p_name.find('P_')
        
        # check if the pins are properly paired
        properly_paired = p_name[:split_ind] + 'N' == n_name[:split_ind+1]
        
        all_properly_paired &= properly_paired
        
        for pin in pin_p, pin_n:
            display(Markdown(f'**{pin["Pin"]}** *{pin["Pin Name"]}* **{pin["Port"]}**'))
        
        if properly_paired:
            display(Markdown('properly paired differential signals'))
        else:
            display(Markdown('WARNING!!! Improperly paired differential signals'))
            
        display(Markdown('--------'))
        
print(f'All properly paired: {all_properly_paired}')

## Differential ports

### Bank 0, 1.8V

--------

### Bank 13, 1.8V

--------

...

properly paired differential signals

--------

### Bank 36, 3.3V

--------

All properly paired: True


### Isolate the DDR3 pins and print those 

In [13]:
display(Markdown(f'## DDR3 Signals'))
for bank_name, pins in banks.items():
    all_pins = pins['single_ended_pins'] + pins['differential_pins']
    DDR3_pins = [pin for pin in all_pins if pin['Port'].startswith('DDR3')]
    
    if not len(DDR3_pins):
        continue
    
    display(Markdown(f'### Bank {bank_name}, {bank_Vs[int(bank_name)]}V'))
    
    for pin in DDR3_pins:        
        display(Markdown(f'**{pin["Pin"]}** *{pin["Pin Name"]}* **{pin["Port"]}**'))

## DDR3 Signals

### Bank 33, 1.35V

**AC4** *IO_L5P_T0_33* **DDR3_A0**

**AD5** *IO_L8N_T1_33* **DDR3_A1**

**AD4** *IO_L5N_T0_33* **DDR3_A2**

...

**AA2** *IO_L9P_T1_DQS_34* **DDR3_UDQS_P**

**AB1** *IO_L9N_T1_DQS_34* **DDR3_UDQS_N**

### Bank 36, 3.3V

**D3** *IO_L24N_T3_36* **DDR3_VTT_S3**

**D4** *IO_L24P_T3_36* **DDR3_VTT_S5**